Подключение Библиотек

In [1]:
import os, gc, time
import numpy as np
import pandas as pd
from pathlib import Path
from typing import List, Optional
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from catboost import CatBoostClassifier, Pool

 Настройки

In [2]:
SEED        = 65
N_SPLITS    = 2
LR          = 0.001
DEPTH       = 2
ITERATIONS  = 100
ES_ROUNDS   = 10
L2_REG      = 1.0
RUN_NAME    = "catboost_cv"

# Укажите путь к вашему train-файлу:
TRAIN_PATH = "train.csv" 

- `SEED = 65` — "магическое число" для воспроизводимости результатов
- `N_SPLITS = 2` — разделим данные на 2 части для валидации
- `LR = 0.01` — скорость обучения
- `DEPTH = 2` — глубина деревьев (компромисс между простотой и сложностью)
- `ITERATIONS = 100` — максимальное количество итераций обучения
- `ES_ROUNDS = 10` — остановимся, если 10 итераций подряд нет улучшения
- `L2_REG = 4.0` — регуляризация против переобучения
- `RUN_NAME` — имя для сохранения файлов

 Вспомогательные функции

Функция для воспроизводимости. Устанавливает одинаковые «случайные» числа при каждом запуске.

In [3]:
def set_seed(seed=SEED):
    import random, os
    random.seed(seed)
    np.random.seed(seed)

In [4]:
set_seed()

Умная функция-детектив. Автоматически находит и сообщает о результате:
- **Целевую переменную** (то, что мы предсказываем): ищет среди "target", "type", "label", "y". Это универсальное решение для разных типов файлов, в задачах поле цели может быть записано по разному, но чаще всего это "target", "type", "label", "y"
- **ID колонку** (идентификатор объектов): ищет среди "object_id", "idx", "id". Так же универсальное решение, так как ключ объекта может быть назван как id, так и "object_id", и "idx",

In [5]:
def detect_columns(df: pd.DataFrame):
    """Определяем имя таргета и id-колонки, если есть."""
    target_candidates = ["target", "type", "label", "y"]
    id_candidates     = ["object_id", "idx", "id"]
    target_col = next((c for c in target_candidates if c in df.columns), None)
    id_col     = next((c for c in id_candidates if c in df.columns), None)
    if target_col is None:
        raise ValueError("Не найдена колонка с таргетом (ожидались: target/type/label/y).")
    return target_col, id_col

Функция-сортировщик. Разделяет колонки на:
- **Числовые** (`num_cols`): возраст, зарплата, температура
- **Категориальные** (`cat_cols`): пол, город, профессия


In [6]:

def get_feature_lists(df: pd.DataFrame, target_col: str, id_col: Optional[str]):
    num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    cat_cols = df.select_dtypes(include=["object","category"]).columns.tolist()
    for col in [target_col, id_col]:
        if col in num_cols: num_cols.remove(col)
        if col in cat_cols: cat_cols.remove(col)
    return num_cols, cat_cols

Загрузка и подготовка данных

In [7]:
df = pd.read_csv(TRAIN_PATH)
target_col, id_col = detect_columns(df) # Загружаем данные и автоматически определяем структуру.

In [8]:
num_cols, cat_cols = get_feature_lists(df, target_col, id_col)
for c in cat_cols:
    df[c] = df[c].astype("category")

1. Определяем типы колонок
2. Явно помечаем категориальные как `category`

In [9]:
# Разделяем фичи/цель
feature_cols = [c for c in df.columns if c not in {target_col, id_col}]
X = df[feature_cols].copy()
y = df[target_col].copy()

Разделяем данные:
- `X` — все признаки (то, по чему предсказываем)
- `y` — целевая переменная (то, что предсказываем)

Создаём список индексов категориальных колонок для CatBoost.

In [10]:
# Индексы категориальных признаков
cat_idx = [X.columns.get_loc(c) for c in cat_cols if c in X.columns]

In [11]:
print(f"Форма данных: X={X.shape}, y={y.shape}. Категориальные: {len(cat_idx)} -> {cat_cols}")

Форма данных: X=(45000, 27), y=(45000,). Категориальные: 1 -> ['quality_flag']


 Кросс-валидация и обучение

In [12]:
# ---------- CV ТРЕНИРОВКА И OOF-ОЦЕНКА ----------
cv = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
oof_pred = np.empty(len(X), dtype=object)
fold_models: List[CatBoostClassifier] = []
fold_scores = []
start_time = time.time()


Подготовка к кросс-валидации:
- `StratifiedKFold` — умное разделение, сохраняющее пропорции классов
- `oof_pred` — массив для "out-of-fold" предсказаний
- `fold_models` — список моделей для каждого фолда
- `fold_scores` — оценки качества по фолдам

Это Как подготовка к экзамену, где мы 5 раз тренируемся на разных частях материала.

In [13]:
params = dict(
    loss_function="MultiClass",
    eval_metric="MultiClass",          # Macro F1 посчитаем сами
    learning_rate=LR,
    depth=DEPTH,
    l2_leaf_reg=L2_REG,
    random_strength=0.8,
    bagging_temperature=0.2,
    auto_class_weights="Balanced",     # балансируем редкие классы
    iterations=ITERATIONS,
    early_stopping_rounds=ES_ROUNDS,
    allow_writing_files=False,
    thread_count=-1,
    random_state=SEED,
    verbose=False
)

Настройки для CatBoost:
- `MultiClass` — многоклассовая классификация
- `auto_class_weights="Balanced"` — автоматически балансирует редкие классы
- `thread_count=-1` — использует все доступные процессоры
- `verbose=False` — не выводит лишнюю информацию

 Цикл обучения по фолдам

In [14]:

for fold, (tr_idx, val_idx) in enumerate(cv.split(X, y), 1):
    X_tr, X_val = X.iloc[tr_idx], X.iloc[val_idx]
    y_tr, y_val = y.iloc[tr_idx], y.iloc[val_idx]
# Для каждого фолда разделяем данные на тренировочную и валидационную части.
    train_pool = Pool(X_tr, y_tr, cat_features=cat_idx)
    valid_pool = Pool(X_val, y_val, cat_features=cat_idx)
# Создаём специальные объекты `Pool` для CatBoost с указанием категориальных признаков.
    model = CatBoostClassifier(**params)
    model.fit(train_pool, eval_set=valid_pool, use_best_model=True)
# 1. Создаём модель с нашими параметрами
# 2. Обучаем её с валидацией и ранней остановкой
    y_val_pred = model.predict(valid_pool).astype(object).ravel()
    score = f1_score(y_val, y_val_pred, average="macro")
    fold_scores.append(score)
    oof_pred[val_idx] = y_val_pred
    fold_models.append(model)
# 1. Делаем предсказания на валидационной части
# 2. Вычисляем F1-score (метрику качества)
# 3. Сохраняем результаты для общей оценки
    print(f"Fold {fold}: Macro F1 = {score:.5f} | best_iter={model.get_best_iteration()}")

Fold 1: Macro F1 = 0.75577 | best_iter=99
Fold 2: Macro F1 = 0.75255 | best_iter=99


Оценка результатов

In [15]:
oof_macro_f1 = f1_score(y, oof_pred, average="macro")
elapsed = time.time() - start_time
print(f"\nOOF Macro F1: {oof_macro_f1:.5f}  (folds: {[f'{s:.4f}' for s in fold_scores]})")
print(f"Время обучения: {elapsed:.1f} c")
# Вычисляем общую оценку качества и выводим результаты.
# Это как честная оценка — каждый объект предсказывается моделью, которая его не видела при обучении.

# Подробный отчёт по каждому классу (precision, recall, F1-score).
print("\nОтчёт по классам (OOF):")
print(classification_report(y, oof_pred, digits=4))

# Матрица ошибок — показывает, какие классы путает модель.:
labels_order = sorted(y.unique().tolist())
cm = pd.DataFrame(confusion_matrix(y, oof_pred, labels=labels_order),
                  index=[f"true_{c}" for c in labels_order],
                  columns=[f"pred_{c}" for c in labels_order])
print("\nConfusion matrix (OOF):")
display(cm)


OOF Macro F1: 0.75410  (folds: ['0.7558', '0.7525'])
Время обучения: 2.2 c

Отчёт по классам (OOF):
                     precision    recall  f1-score   support

exoplanet_candidate     0.0192    0.1130    0.0329       230
             galaxy     0.7175    0.9583    0.8206      3380
 main_sequence_star     0.9930    0.8988    0.9435     29000
             quasar     0.9037    0.9443    0.9236      1580
          red_giant     0.9657    0.9103    0.9372      7880
        white_dwarf     0.7671    0.9962    0.8668      2930

           accuracy                         0.9092     45000
          macro avg     0.7277    0.8035    0.7541     45000
       weighted avg     0.9447    0.9092    0.9228     45000


Confusion matrix (OOF):


,pred_exoplanet_candidate,pred_galaxy,pred_main_sequence_star,pred_quasar,pred_red_giant,pred_white_dwarf
true_exoplanet_candidate,26,0,169,0,0,35
true_galaxy,0,3239,0,129,8,4
true_main_sequence_star,1324,566,26066,13,247,784
true_quasar,0,58,0,1492,0,30
true_red_giant,1,650,16,7,7173,33
true_white_dwarf,0,1,0,10,0,2919


Финальная модель

In [1]:
# ---------- ФИНАЛЬНАЯ МОДЕЛЬ НА ВСЁМ TRAIN ----------
# Среднее по лучшим итерациям фолдов для устойчивости
best_iters = [m.get_best_iteration() or ITERATIONS for m in fold_models]
final_iters = int(np.clip(np.mean(best_iters), 200, ITERATIONS))
final_params = {**params, "iterations": final_iters, "early_stopping_rounds": None, "verbose": False}

final_model = CatBoostClassifier(**final_params)
final_pool  = Pool(X, y, cat_features=cat_idx)
final_model.fit(final_pool)

final_model.save_model(f"{RUN_NAME}_model.cbm")
pd.Series(final_model.get_feature_importance(prettified=False), index=feature_cols)\
  .sort_values(ascending=False)\
  .head(25)\
  .to_csv(f"{RUN_NAME}_top_features.csv")

print(f"\nФинальная модель обучена на всём train. iterations={final_iters}.")
print(f"Сохранено: {RUN_NAME }_model.cbm и {RUN_NAME}_top_features.csv")

NameError: name 'fold_models' is not defined